In [ ]:
import numpy as np
import DnnLib

data_train = np.load("../data/fashion_train.npz")
X_train = data_train["images"].astype(np.float64).reshape(len(data_train["images"]), -1) / 255.0
y_train = np.eye(10)[data_train["labels"]]

data_test = np.load("../data/fashion_test.npz")
X_test = data_test["images"].astype(np.float64).reshape(len(data_test["images"]), -1) / 255.0
y_test = np.eye(10)[data_test["labels"]]
print("Datos Cargados")

Datos Cargados


In [2]:
layers = [
    DnnLib.DenseLayer(784, 128, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(128, 64, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(64, 10, DnnLib.ActivationType.SOFTMAX)
]

In [3]:
optimizer = DnnLib.Adam(learning_rate=0.001)


In [4]:
def train_minibatch(layers, optimizer, X, y, batch_size=64, epochs=10):
    n_samples = X.shape[0]
    for epoch in range(epochs):
        indices = np.random.permutation(n_samples)
        X_shuff, y_shuff = X[indices], y[indices]
        epoch_loss, n_batches = 0.0, 0
        
        for i in range(0, n_samples, batch_size):
            X_batch = X_shuff[i:i+batch_size]
            y_batch = y_shuff[i:i+batch_size]

            # Forward
            activation = X_batch
            for layer in layers:
                activation = layer.forward(activation)
            output = activation

            # Loss + grad
            loss = DnnLib.cross_entropy(output, y_batch)
            grad = DnnLib.cross_entropy_gradient(output, y_batch)

            # Backward
            for layer in reversed(layers):
                grad = layer.backward(grad)
                optimizer.update(layer)

            epoch_loss += loss
            n_batches += 1

        print(f"Epoch {epoch}, Loss: {epoch_loss/n_batches:.4f}")


In [19]:
print("Iniciando Entrenamiento")
train_minibatch(layers, optimizer, X_train, y_train, batch_size=64, epochs=10)
print("Datos Entrenados")

Iniciando Entrenamiento
Epoch 0, Loss: 0.2317
Epoch 1, Loss: 0.2226
Epoch 2, Loss: 0.2189
Epoch 3, Loss: 0.2093
Epoch 4, Loss: 0.2032
Epoch 5, Loss: 0.1980
Epoch 6, Loss: 0.1927
Epoch 7, Loss: 0.1866
Epoch 8, Loss: 0.1808
Epoch 9, Loss: 0.1744
Datos Entrenados


In [20]:
preds = X_test
for layer in layers:
    preds = layer.forward(preds)
    

pred_classes = np.argmax(preds, axis=1)
true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(pred_classes == true_classes)
print("Accuracy en test:", accuracy)


Accuracy en test: 0.8883


In [18]:
import json

def save_model(layers, filename="model.json"):
    model_data = []
    for layer in layers:
        if isinstance(layer, DnnLib.DenseLayer):
            model_data.append({
                "type": "DenseLayer",
                "input_dim": layer.weights.shape[1],
                "output_dim": layer.weights.shape[0],
                "activation": layer.activation_type.name,
                "weights": layer.weights.tolist(),
                "bias": layer.bias.tolist()
            })
    with open(filename, "w") as f:
        json.dump(model_data, f)
    print(f"Modelo guardado en {filename}")
    
save_model(layers,"../modelos/testmodel.json")

Modelo guardado en ../modelos/testmodel.json


In [ ]:
def load_model(filename="model.json"):
    import json
    import numpy as np
    import DnnLib

    with open(filename, "r") as f:
        model_data = json.load(f)

    layers = []
    for layer_info in model_data:
        if layer_info["type"] == "DenseLayer":
            layer = DnnLib.DenseLayer(
                layer_info["input_dim"],
                layer_info["output_dim"],
                getattr(DnnLib.ActivationType, layer_info["activation"])
            )            
            layer.weights[:] = np.array(layer_info["weights"], dtype=np.float64)
            layer.bias[:] = np.array(layer_info["bias"], dtype=np.float64)
            
            if hasattr(layer, 'cache'):
                layer.cache = None
            if hasattr(layer, 'grad_cache'):
                layer.grad_cache = None
            
            layers.append(layer)

    print(f"Modelo cargado desde {filename}")
    return layers


# Cargar modelo
#loaded_layers = load_model("../modelos/model.json")

# Forward sobre el test set
#preds = X_test
#for layer in loaded_layers:
#    preds = layer.forward(preds)

#true_classes = np.argmax(y_test, axis=1)
#acc = np.mean(pred_classes == true_classes)
#print("Accuracy del modelo cargado:", acc)
